## Parsing Sales Alerts

This notebook is used to parse json like sales alerts data
    

#### Importing Libraries

In [111]:
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [168]:
df_sales = pd.read_csv('sales_alerts_2022_multithread.csv')
df_sales.head()

,week,userid,data
0,2022-12-25,cxb9ke5,"{'notification_count': 2, 'sales_alert': [{'DE..."
1,2022-12-18,cxb9ke5,"{'notification_count': 1, 'sales_alert': [{'DE..."
2,2022-08-14,cxb9ke5,"{'notification_count': 1, 'sales_alert': [{'DE..."
3,2022-07-10,cxb9ke5,"{'notification_count': 2, 'sales_alert': [{'DE..."
4,2022-05-08,cxb9ke5,"{'notification_count': 1, 'sales_alert': [{'DE..."


In [169]:
df_sales.shape

(394928, 3)

In [170]:
pd.DataFrame(df_sales.data)[0:10]

,data
0,"{'notification_count': 2, 'sales_alert': [{'DE..."
1,"{'notification_count': 1, 'sales_alert': [{'DE..."
2,"{'notification_count': 1, 'sales_alert': [{'DE..."
3,"{'notification_count': 2, 'sales_alert': [{'DE..."
4,"{'notification_count': 1, 'sales_alert': [{'DE..."
5,"{'notification_count': 1, 'sales_alert': [{'DE..."
6,"{'notification_count': 1, 'sales_alert': [{'DE..."
7,"{'notification_count': 2, 'sales_alert': [{'DE..."
8,"{'notification_count': 1, 'sales_alert': [{'DE..."
9,"{'notification_count': 1, 'sales_alert': [{'DE..."


#### Breaking down the alerts into two main columns `notification_count` and `sales_alert`

In [171]:
df_sales['notification_count'] = df_sales.apply(lambda row: eval(row['data'])['notification_count'],
                     axis=1)

In [172]:
df_sales['sales_alert'] = df_sales.apply(lambda row: eval(row['data'])['sales_alert'],
                     axis=1)

#### Parsing sales alert column

In [173]:
df_sales = df_sales.explode('sales_alert')


In [174]:
df_sales.head()

,week,userid,data,notification_count,sales_alert
0,2022-12-25,cxb9ke5,"{'notification_count': 2, 'sales_alert': [{'DE...",2,"{'DECREASE': True, 'ANOMALY_DURATION': 2, 'SKU..."
0,2022-12-25,cxb9ke5,"{'notification_count': 2, 'sales_alert': [{'DE...",2,"{'DECREASE': True, 'ANOMALY_DURATION': 2, 'SKU..."
1,2022-12-18,cxb9ke5,"{'notification_count': 1, 'sales_alert': [{'DE...",1,"{'DECREASE': True, 'ANOMALY_DURATION': 2, 'SKU..."
2,2022-08-14,cxb9ke5,"{'notification_count': 1, 'sales_alert': [{'DE...",1,"{'DECREASE': False, 'ANOMALY_DURATION': 2, 'SK..."
3,2022-07-10,cxb9ke5,"{'notification_count': 2, 'sales_alert': [{'DE...",2,"{'DECREASE': True, 'ANOMALY_DURATION': 2, 'SKU..."


In [120]:
df_sales['sales_alert']=[{**dc,**{'week':w}, **{'userid':u}, **{'notification_count':notif_cnt}} for dc,w,u,notif_cnt in\
                         zip(df_sales['sales_alert'],df_sales['week'],df_sales['userid'],df_sales['notification_count'])]
df_sales = pd.DataFrame(df_sales['sales_alert'].tolist())

In [178]:
df_sales.to_csv('parsed_sales.csv',index=False)

/Users/ET92ZFW/Projects/1_PromoAnalytics/venv-promo/lib/python3.9/site-packages/pandas/core/internals/blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)

KeyboardInterrupt



In [199]:
#Testing the code
df_test = pd.read_csv('parsed_sales.csv')

In [200]:
df_test

,WEEK,USER_ID,NOTIFICATION_COUNT,DECREASE,ANOMALY_DURATION,SKU_CRT_DT,NOTIFICATION_METADATA,SKU_NBR,DIFFERENCE_PERC,DOLLAR_DIFF,SKU_DESC
0,2022-12-25,cxb9ke5,2,True,2,NaN,[],1004809670,-61.151443,-25494.11,1004809670-6' FG STEP 300LB IA
1,2022-12-25,cxb9ke5,2,True,2,NaN,"[{'STORE_ANOMALY': [{'STORE_RANK': 2, 'DIFFERE...",1004809582,-37.511455,-170769.56,1004809582-22\ X 54\ AL ATTIC 8-10' 375LB. R5
2,2022-12-18,cxb9ke5,1,True,2,NaN,[],1002101011,-113.458332,-22550.32,1002101011-40 FT. FIBERGLASS EXTENSION LADDER W
3,2022-08-14,cxb9ke5,1,False,2,NaN,"[{'MKT_NBR': 16, 'STORE_ANOMALY': [{'STORE_RAN...",1004809670,87.831441,14537.50,1004809670-6' FG STEP 300LB IA
4,2022-07-10,cxb9ke5,2,True,2,NaN,"[{'STORE_ANOMALY': [{'STORE_RANK': 1, 'DIFFERE...",1007170561,-30.820028,-24791.80,1007170561-22\ X 54\ 8'-10' AL FRAME ATTIC 375#
...,...,...,...,...,...,...,...,...,...,...,...
1904993,2022-02-27,txe9771,2,False,2,NaN,"[{'STORE_ANOMALY': [{'STORE_RANK': 1, 'DIFFERE...",1003950578,106.622857,9898.93,1003950578-1 CU FT COW MANURE & ORGANIC COMPOST
1904994,2022-01-02,txe9771,2,False,2,NaN,[],1006844973,57.165669,6150.38,1006844973-BONNIE HARVEST SELECT RAISEDBED SOIL
1904995,2022-01-02,txe9771,2,True,5,NaN,[],1000716974,-99.454350,-97835.78,1000716974-MIRACLE GRO CHRISTMAS TREE
1904996,2022-10-23,rxl93r1,1,False,2,NaN,[],1005617764,177.976083,18244.88,1005617764-2.5QT GRAY W/ HANDLE


In [201]:
df_test.shape

(1904998, 11)

In [202]:
df_test.columns

Index(['WEEK', 'USER_ID', 'NOTIFICATION_COUNT', 'DECREASE', 'ANOMALY_DURATION',
       'SKU_CRT_DT', 'NOTIFICATION_METADATA', 'SKU_NBR', 'DIFFERENCE_PERC',
       'DOLLAR_DIFF', 'SKU_DESC'],
      dtype='object')

In [203]:
df_test = df_test.loc[df_sales.astype(str).drop_duplicates().index]

In [204]:
df_test.shape

(1904998, 11)

In [205]:
df_test.groupby(['WEEK','USER_ID','SKU_NBR']).size().sort_values(ascending=False)

WEEK        USER_ID  SKU_NBR   
2022-01-02  4922bm   1000341904    1
2022-10-23  txl9a5z  1000204236    1
            txl9fgs  1005183419    1
                     1005183351    1
            txl9e0m  1006438411    1
                                  ..
2022-07-31  txh9ih0  1007473578    1
                     1007430345    1
                     1006703282    1
                     1006225359    1
2022-12-25  zxz9ly7  1007762421    1
Length: 1904998, dtype: int64